In [63]:
import pandas as pd
import re as re

pd.set_option('display.max_columns', 500)

In [64]:
# Take in Excel (Windows)
"""
itemized = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", header=2, sheet_name=0)
invoice_data = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", sheet_name=1)
qbo = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/QBO_customers(1).xlsx")
amt = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name='AMT')
gp_acoustics = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name='GPAcoustics')

"""

'\nitemized = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", header=2, sheet_name=0)\ninvoice_data = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Invoice_data.xlsx", sheet_name=1)\nqbo = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/QBO_customers(1).xlsx")\namt = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name=\'AMT\')\ngp_acoustics = pd.read_excel("C:/Users/danie/OneDrive/Projects/fedex_reconciliation/Exensiv.xlsx", sheet_name=\'GPAcoustics\')\n\n'

In [65]:
# Take in Excel (MAC)
itemized = pd.read_excel(
    "/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Nautical 11-2024.xlsx",
    header=2,
    sheet_name=0,
)
invoice_data = pd.read_excel(
    "/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Nautical 11-2024.xlsx",
    sheet_name=1,
)
qbo = pd.read_excel(
    "/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/QBO_customers.xlsx"
)
amt = pd.read_excel(
    "/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Exensiv.xlsx",
    sheet_name="AMT",
)
gp_acoustics = pd.read_excel(
    "/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Exensiv.xlsx",
    sheet_name="GPAcoustics",
)

# **Prepare DF**

In [66]:
# Add Source Column

invoice_data['Source'] = 'Invoice Data'
qbo['Source'] = 'QBO'
amt['Source'] = 'AMT'
gp_acoustics['Source'] = 'GP Acoustics'

In [67]:
df = pd.concat(objs=[invoice_data,qbo, amt, gp_acoustics],join='outer')

In [68]:
amt = df[df['CustomerIdentifier.Name'] == 'AMT']
gp_acoustics = df[df['CustomerIdentifier.Name'] == 'GP Acoustics']
qbo = df[df['Source'] == 'QBO']
invoice_data = df[df['Source'] == 'Invoice Data']

In [69]:
df['Source'].unique()

array(['Invoice Data', 'QBO', 'AMT', 'GP Acoustics'], dtype=object)

In [70]:
amt = amt.dropna(axis=1, how='all')
gp_acoustics = gp_acoustics.dropna(axis=1, how='all')
qbo = qbo.dropna(axis=1, how='all')
invoice_data = invoice_data.dropna(axis=1, how='all')

# **Invoice Data** <-> **QBO**

## - Compare **Invoice Data** [`Customer PO #`] to **QBO** [`Display_Name`]

In [71]:
def compare_qbo(qbo: pd.DataFrame, invoice_data: pd.DataFrame) -> pd.DataFrame:
    """
    Function: Compares FedEx invoice with QuickBooks via keys 'Customer PO #' and 'Display_Name'
    Input: Original QuickBooks and FedEx Invoice file
    Output: Pandas DataFrame with values not found in QuickBooks
    """

    # ? is 'Display_Name' the only key to compare against?

    qbo_found = pd.merge(
        qbo, invoice_data, right_on="Customer PO #", left_on="Display_Name", how="inner", suffixes=['_qbo','_invoice_data']
    )

    lst = set()
    for i in invoice_data["Customer PO #"]:
        if i not in list(qbo_found["Display_Name"].unique()):
            lst.add(i)

    qbo_not_found = pd.DataFrame()
    qbo_not_found["Customer PO #"] = pd.DataFrame(lst)
    qbo_not_found = qbo_not_found.merge(
        invoice_data,
        on="Customer PO #",
        how="left",
    )
    return qbo_found, qbo_not_found

In [72]:
# Run it - Customer PO's found in QBO and not found in QBO

qbo_found, invoice_data_not_qbo = compare_qbo(qbo, invoice_data)

In [110]:
# 6 PO's found in QBO

print(f'unique: {qbo_found['Display_Name'].nunique()}',
      f'total: {len(qbo_found)}')

unique: 6 total: 2218


In [107]:
# 83 Not found in QBO

print(f'unique: {invoice_data_not_qbo['Customer PO #'].nunique()}',
                 f'total: {len(invoice_data_not_qbo)}')


unique: 56 total: 4973


# **Invoice Data** [`Reference`] <-> **Extensiv** [`Reference`]

## - For all **Invoice Data** [`Customer PO #`] not in **QBO** [`Display Name`]

### - Add RegEx column to `not_found` with [Reference] patterns

In [75]:
def add_pattern_column(invoice_data: pd.DataFrame) -> pd.DataFrame:
    """
    Function: Adds a column to the FedEx Invoice DataFrame with the RegEx pattern symbolizing 'Customer PO #'
    Input: FedEx Invoice DataFrame
    Output: DataFrame with added 'Pattern' column
    """

    def reg_tokenizer(value):

        with_letters = re.sub(r"[a-zA-Z]+", r"\\w+", str(value))
        with_numbers = re.sub(r"\d+", r"\\d+", with_letters)
        with_spaces = re.sub(r"\s+", r"\\s+", with_numbers)

        final = re.compile(with_spaces)

        return final

    token_lst = []

    for i in invoice_data["Reference"]:
        token_lst.append(reg_tokenizer(i))

    invoice_data["Pattern"] = token_lst

    return invoice_data

In [76]:
invoice_data_not_qbo = add_pattern_column(invoice_data)

### - Compare all `not_found`[Reference], `not_found`[Pattern] values to Extensiv Table

In [77]:
def find_extensiv_reference_columns(extensiv_table: pd.DataFrame, invoice_data_w_patterns: pd.DataFrame) -> dict:  # fmt: skip
    """
    Function: Finds all of the columns in the Extensiv table that match each 'Reference' in FedEx Invoice not in QBO
    Input: Extensiv DataFrame, FedEx Invoice DataFrame w/ added 'Pattern' column
    Ouput: Dictionary {'match_lst': list of Extensiv columns that match 'Reference' pattern,
                       'Total Charges': Charges associated with that 'Reference' in FedEx Invoice,
                       'Tracking #': Tracking number associated with that 'Reference' in FedEx Invoice}
    Notes: May not need Total Charges and Tracking # in the end
    """

    def find_col_match(extensiv_table: pd.DataFrame, ref_pattern: pd.Series) -> list:
        """
        Function: Subfunction to iterate through each of the patterns in FedEx Invoice
        Input: Extensiv DataFrame, FedEx Reference patterns as a Series in a for loop
        Ouput: List of columns that match given Reference pattern
        """
        col_lst = set()

        for col in extensiv_table.columns:

            for value in extensiv_table[col]:

                if isinstance(value, float) and value.is_integer():
                    
                    value = int(value)

                if re.fullmatch(ref_pattern, str(value)):

                    col_lst.add(col)
                    break

                else:
                    break

        if len(col_lst) != 0:
            return col_lst
        else:
            return None

    match_dct = dict()
    suffix = 0

    for i, v in enumerate(invoice_data_w_patterns["Reference"]):

        if i != 0 and v == invoice_data_w_patterns["Reference"][i - 1]:

            suffix += 1
            v = f"{v}-s{suffix}"

        elif i != 0 and v != invoice_data_w_patterns["Reference"][i - 1]:
            suffix = 0
        else:
            continue

        match_lst = find_col_match(
            extensiv_table, invoice_data_w_patterns["Pattern"][i]
        )

        if match_lst is not None and not pd.isna(v):

            match_dct[v] = {
                "match_lst": match_lst,
                "Total Charges": invoice_data_w_patterns["Total Charges"][i],
                "Tracking #": invoice_data_w_patterns["Tracking #"][i],
            }

    return match_dct

In [117]:
gp_reference_columns = find_extensiv_reference_columns(gp_acoustics, invoice_data_not_qbo)
amt_reference_columns = find_extensiv_reference_columns(amt, invoice_data_not_qbo)

In [79]:
gp_reference_columns

{10554: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'LastModifiedByIdentifier.Id',
   'LoadedState',
   'NumUnits1',
   'OrderId',
   'Packages.CartonId',
   'Packages.Cod',
   'Packages.CodAmount',
   'Packages.Height',
   'Packages.InsuredAmount',
   'Packages.Oversize',
   'Packages.PackageId',
   'Packages.PackageTypeId',
   'Packages.Ucc128',
   'Packages.Width',
   'ParcelLabelType',
   'ReferenceNum',
   'RouteCandidate',
   'RoutingInfo.BillOfLading',
   'RoutingInfo.Mode',
   'RoutingInfo.TrackingNumber',
   'ShipTo.AddressStatus',
   'ShipTo.ContactId',
   'ShipTo.Zip',
   'Status',
   'TransactionEntryType',
   'WarehouseTransactionSourceType'},
  'Total Charges': 6.54,
  'Tracking #': 280881245477.0},
 10555: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'LastModifiedByIdentifier.I

In [96]:
def find_value_match(extensiv_table: pd.DataFrame, reference_matches: dict) -> list:

    match_lst = list()

    for reference in reference_matches:

        matches = reference_matches[reference]["match_lst"]
        total_charges = reference_matches[reference]["Total Charges"]
        tracking_number = reference_matches[reference]["Tracking #"]

        for col in extensiv_table[list(matches)]:

            for i, val in enumerate(extensiv_table[col]):

                base_reference = re.sub(r"-s\d+$", "", str(reference))

                if val == reference or val == base_reference:

                    match_entry = {
                        "Reference": base_reference,
                        "Name": extensiv_table["CustomerIdentifier.Name"][i],
                        "Column": col,
                        "Total Charges": total_charges,
                        "Tracking #": tracking_number,
                    }

                    if match_entry not in match_lst:
                        match_lst.append(match_entry)

    if not match_lst:
        print(f"No Matches")

    else:
        return match_lst

In [118]:
gp_reference_matches = find_value_match(gp_acoustics,gp_reference_columns)
amt_reference_matches = find_value_match(amt, amt_reference_columns)

No Matches


In [119]:
amt_reference_matches

# **Invoice Data** [`Receiver Info`] <-> **Extensiv** [`Receiver Info`]

### - Create **Receiver Info** Dicts

In [83]:
def create_extensiv_receiver_info(extensiv_table: pd.DataFrame) -> dict:

    extensiv_receiver_info = extensiv_table[
        [
            "ShipTo.CompanyName",
            "ShipTo.Name",
            "ShipTo.Address1",
            "CustomerIdentifier.Name",
        ]
    ]

    extensiv_receiver_info_nd = extensiv_receiver_info.drop_duplicates(
        [
            "ShipTo.CompanyName",
            "ShipTo.Name",
            "ShipTo.Address1",
            "CustomerIdentifier.Name",
        ]
    )

    extensiv_receiver_dct = dict()

    for i, row in extensiv_receiver_info_nd.iterrows():

        extensiv_receiver_dct[i] = {
            "Receiver Address": row["ShipTo.Address1"],
            "Receiver Company": row["ShipTo.CompanyName"],
            "Receiver Name": row["ShipTo.Name"],
            "Customer Identifier": row["CustomerIdentifier.Name"],
        }

    return extensiv_receiver_dct

In [89]:
def create_invoice_data_receiver_info(invoice_data: pd.DataFrame) -> dict:  # fmt: skip

    invoice_data_dct = {}

    for i, row in invoice_data.iterrows():

        invoice_data_dct[i] = {
            "Receiver Address": row["Receiver Address"],
            "Receiver Company": row["Receiver Company"],
            "Receiver Name": row["Receiver Name"],
            "Tracking #": row["Tracking #"],
        }

    return invoice_data_dct

In [120]:
# Run it

gp_receiver_info = create_extensiv_receiver_info(gp_acoustics)
amt_receiver_info = create_extensiv_receiver_info(amt)
invoice_data_receiver_info = create_invoice_data_receiver_info(invoice_data)

### Compare **Invoice Data** [`Receiver Info`] and **Extensiv** [`Receiver Info`]

In [130]:
def compare_receiver_info(invoice_data_receiver_info: dict, extensiv_receiver_info: dict) -> list:  # fmt: skip

    match_entry = dict()
    match_lst = list()

    for i in invoice_data_receiver_info:

        for e in extensiv_receiver_info:

            if (
                invoice_data_receiver_info[i]["Receiver Address"]
                == extensiv_receiver_info[e]["Receiver Address"]
                or invoice_data_receiver_info[i]["Receiver Name"]
                == extensiv_receiver_info[e]["Receiver Name"]
                or invoice_data_receiver_info[i]["Receiver Company"]
                == extensiv_receiver_info[e]["Receiver Company"]
            ):

                match_entry = {
                    "Address": invoice_data_receiver_info[i]["Receiver Address"],
                    "Name": invoice_data_receiver_info[i]["Receiver Name"],
                    "Company": invoice_data_receiver_info[i]["Receiver Company"],
                    "Customer": extensiv_receiver_info[e]["Customer Identifier"],
                }

                if match_entry not in match_lst:
                    match_lst.append(match_entry)

  
    return match_lst


In [131]:
gp_receiver_matches = compare_receiver_info(invoice_data_receiver_info, gp_receiver_info)
amt_receiver_matches = compare_receiver_info(invoice_data_receiver_info, amt_receiver_info)

In [132]:
gp_receiver_matches

[{'Address': '8 THORNTON RD',
  'Name': 'MUSICAL FULFILMENT SERVICES',
  'Company': 'MUSICAL FULFILMENT SERVICES',
  'Customer': 'GP Acoustics'},
 {'Address': '180 CARPENTER AVE',
  'Name': 'ALTO MUSIC',
  'Company': 'ALTO MUSIC',
  'Customer': 'GP Acoustics'},
 {'Address': '7091 NW 51ST ST',
  'Name': 'MIAMI PARTS IMPORT,INC',
  'Company': 'MIAMI PARTS IMPORT,INC',
  'Customer': 'GP Acoustics'},
 {'Address': '13 MONMOUTH ST',
  'Name': 'RELIC MUSIC LLC',
  'Company': 'RELIC MUSIC LLC',
  'Customer': 'GP Acoustics'},
 {'Address': '403 S H ST',
  'Name': 'ADI TEJADA',
  'Company': 'ADI TEJADA',
  'Customer': 'GP Acoustics'},
 {'Address': '1229 TRUMAN RD',
  'Name': 'PAOLO GREGOLETTO',
  'Company': 'PAOLO GREGOLETTO',
  'Customer': 'GP Acoustics'},
 {'Address': '28 SUNDANCE CIR',
  'Name': 'ATLAS CUSTOM CABINETS',
  'Company': 'ATLAS CUSTOM CABINETS',
  'Customer': 'GP Acoustics'},
 {'Address': '71 STEEP RD',
  'Name': 'STEVEN MILLER',
  'Company': 'STEVEN MILLER',
  'Customer': 'GP Acou

In [133]:
amt_reference_matches

In [139]:
def make_final_df(reference_matches, receiver_matches, invoice_data_not_qbo):

    try:

        final_matches_lst = []
        final_matches_lst.extend(reference_matches)
        final_matches_lst.extend(receiver_matches)

        for i, row in invoice_data_not_qbo.iterrows():

            for dct in final_matches_lst:
                
                if 'Reference' in dct and dct['Reference'] == row['Reference']:
                    invoice_data_not_qbo.loc[i, 'Customer PO #'] = dct['Name']
                elif 'Address' in dct and dct['Address'] == row['Receiver Address']:
                    invoice_data_not_qbo.loc[i, 'Customer PO #'] = dct['Customer']
                elif 'Name' in dct and dct['Name'] == row['Receiver Name']:
                    invoice_data_not_qbo.loc[i, 'Customer PO #'] = dct['Customer']
                elif 'Company' in dct and dct['Company'] == row['Receiver Company']:
                    invoice_data_not_qbo.loc[i, 'Customer PO #'] = dct['Customer']
                elif 'Reference' in dct:
                    try:
                        dct['Reference'] = int(dct['Reference'])
                    except ValueError:
                        pass
    except TypeError:
        pass

    return invoice_data_not_qbo

In [140]:
final_df = make_final_df(gp_reference_matches, gp_receiver_matches, invoice_data_not_qbo)
final_df = make_final_df(amt_reference_matches, amt_receiver_matches, invoice_data_not_qbo)

In [141]:
final_df

,Shipper #,Invoice #,Invoice Date,Invoice Amount,Tracking #,Ship Date,Delivery Date,Delivery Time,Service Level,Zone,Reference,Department,Bill Option,Actual Weight,Bill Weight,Shipper Name,Shipper Company,Shipper Address,Shipper City,Shipper State,Shipper Postal Code,Shipper Country,Receiver Name,Receiver Company,Receiver Address,Receiver City,Receiver State,Receiver Postal Code,Receiver Country,Customer PO #,Declared Value,Customs Value,Shipper Account,Reference 2,Length,Width,Height,Control #,Published Amount,Discounted Amount,Residential,DAS,Fuel,Saturday Delivery,Add'd Handling,Misc. Charges,Client,Coding,Total Charges,Adjusted Amount,Audited Amount,Service Packaging,Source,Pattern
0,693070511.0,866891360.0,2024-11-01,24182.11,2.808813e+11,2024-10-21,2024-10-23,11:33:00,Ground,4.0,10548,NaN,Prepaid,7.0,8.0,Nautical - Renner,Nautical - Renner,16100 W. 116th St,Lenexa,KS,66219.0,US,MUSICAL FULFILLMENT MS,MUSICAL FULFILLMENT MS,12914 STATELINE RD,OLIVE BRANCH,MS,386544017.0,US,700662646/SD264034,0.0,0.0,693070511.0,NaN,14.0,14.0,14.0,ADVA4452,16.50,-12.01,0.00,0.0,0.36,0.0,0.0,0.00,Nautical,59062/587120,4.85,0.0,4.85,Customer Packaging,Invoice Data,re.compile('\\d+')
1,693070511.0,866891360.0,2024-11-01,24182.11,2.808812e+11,2024-10-21,2024-10-23,11:33:00,Ground,4.0,10554,NaN,Prepaid,24.1,25.0,Nautical - Renner,Nautical - Renner,16100 W. 116th St,Lenexa,KS,66219.0,US,MUSICAL FULFILLMENT MS,MUSICAL FULFILLMENT MS,12914 STATELINE RD,OLIVE BRANCH,MS,386544017.0,US,GP Acoustics,0.0,0.0,693070511.0,NaN,20.0,14.0,14.0,ADVA4452,23.16,-17.11,0.00,0.0,0.49,0.0,0.0,0.00,Nautical,59062/587120,6.54,0.0,6.54,Customer Packaging,Invoice Data,re.compile('\\d+')
2,693070511.0,866891360.0,2024-11-01,24182.11,2.808814e+11,2024-10-21,2024-10-23,11:33:00,Ground,4.0,10555,NaN,Prepaid,20.5,21.0,Nautical - Renner,Nautical - Renner,16100 W. 116th St,Lenexa,KS,66219.0,US,MUSICAL FULFILLMENT MS,MUSICAL FULFILLMENT MS,12914 STATELINE RD,OLIVE BRANCH,MS,386544017.0,US,GP Acoustics,0.0,0.0,693070511.0,NaN,14.0,14.0,14.0,ADVA4452,20.73,-15.32,0.00,0.0,0.44,0.0,0.0,0.00,Nautical,59062/587120,5.85,0.0,5.85,Customer Packaging,Invoice Data,re.compile('\\d+')
3,693070511.0,866891361.0,2024-11-01,21136.17,2.808815e+11,2024-10-21,2024-10-23,12:17:00,Ground,4.0,10556,NaN,Prepaid,12.5,13.0,Nautical - Renner,Nautical - Renner,16100 W. 116th St,Lenexa,KS,66219.0,US,MUSICAL FULFILLMENT OH,MUSICAL FULFILLMENT OH,1040 N WYNN RD,OREGON,OH,436161430.0,US,GP Acoustics,0.0,0.0,693070511.0,NaN,14.0,14.0,14.0,ADVA4452,17.43,-12.88,0.00,0.0,0.37,0.0,0.0,0.00,Nautical,59062/587120,4.92,0.0,4.92,Customer Packaging,Invoice Data,re.compile('\\d+')
4,693070511.0,866891361.0,2024-11-01,21136.17,2.808818e+11,2024-10-21,2024-10-24,12:23:00,Ground,6.0,10557,NaN,Prepaid,10.9,11.0,Nautical - Renner,Nautical - Renner,16100 W. 116th St,Lenexa,KS,66219.0,US,MUSICAL FULFILLMENT NV,MUSICAL FULFILLMENT NV,450 MAESTRO DR,RENO,NV,895111294.0,US,GP Acoustics,0.0,0.0,693070511.0,NaN,14.0,14.0,14.0,ADVA4452,19.63,-14.50,0.00,0.0,0.42,0.0,0.0,0.00,Nautical,59062/587120,5.55,0.0,5.55,Customer Packaging,Invoice Data,re.compile('\\d+')
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4968,693070511.0,869086366.0,2024-11-22,39212.48,4.181323e+11,2024-10-22,2024-11-06,12:10:00,Ground,5.0,RECESS-22124-00020,NaN,Prepaid,29.1,30.0,Nautical Fulfillment,Nautical Fulfillment,16100 W 116th St,Lenexa,KS,66219.0,US,Nic,My Gym Childrens Fitness Center -,"302 Harry S. Truman Parkway, Suite",Annapolis,MD,21401.0,US,Nautical/Recess/OxiClean,0.0,0.0,693070511.0,NaN,17.0,17.0,17.0,ADVA4472,32.07,-23.70,0.00,0.0,0.68,0.0,0.0,0.00,Nautical,59062/587120,9.05,0.0,9.05,Customer Packaging,Invoice Data,re.compile('\\w+-\\d+-\\d+')
4969,693070511.0,869086369.0,2024-11-22,25820.77,4.181324e+11,2024-11-12,2024-11-15,12:14:00,Ground,6.0,Whill 198446,NaN,Prepaid,